<a href="https://colab.research.google.com/github/sana-f-shah/Smart-Meter-Analysis/blob/main/notebooks/xgb_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import os

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from xgboost import XGBRegressor

data_dir = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/data_versions/regression'
model_dir = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/models/regression'

dataset_files = {
    'original': f'{data_dir}/original.parquet',
    'no_outliers': f'{data_dir}/no_outliers.parquet',
    'winsorized': f'{data_dir}/winsorized.parquet',
    'imputed': f'{data_dir}/imputed.parquet'
}

target = 'total_consumption_active_import'

results = {
    'dataset': [],
    'rmse': [],
    'mae': [],
    'mape (%)': [],
    'median_ae': [],
    'r2': [],
    'mbe (bias)': []
}

for name, path in dataset_files.items():
    print(f'Processing dataset: {name}')
    df = pd.read_parquet(path)
    df = df.drop(columns='data_collection_log_timestamp')

    y = df[target]
    X = df.drop(columns=[target])

    df_model = pd.concat([X, y], axis=1).dropna()
    X = df_model.drop(columns=target)
    y = df_model[target]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model = XGBRegressor(
        objective='reg:squarederror',
        tree_method='hist',
        device='cuda',
        predictor='gpu_predictor',
        n_estimators=100,
        verbosity=0,
        random_state=42
    )
    model.fit(X_train, y_train)

    model_path = os.path.join(model_dir, f'{name}_model.json')
    model.save_model(model_path)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    medae = median_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mbe = np.mean(y_pred - y_test)

    non_zero_mask = y_test != 0
    if non_zero_mask.any():
        mape = np.mean(np.abs((y_test[non_zero_mask] - y_pred[non_zero_mask]) / y_test[non_zero_mask])) * 100
    else:
        mape = np.nan

    results['dataset'].append(name)
    results['rmse'].append(rmse)
    results['mae'].append(mae)
    results['mape (%)'].append(mape)
    results['median_ae'].append(medae)
    results['r2'].append(r2)
    results['mbe (bias)'].append(mbe)

val_results_df = pd.DataFrame(results)
print('\nModel Performance Summary:')
print(val_results_df)

Processing dataset: original
Processing dataset: no_outliers
Processing dataset: winsorized
Processing dataset: imputed

Model Performance Summary:
       dataset        rmse        mae  mape (%)  median_ae        r2  \
0     original  298.042478  58.606051  0.713277  20.001953  0.998690   
1  no_outliers   38.504950  23.120098  0.554356  15.162598  0.999947   
2   winsorized  115.548883  41.638365  0.452491  17.974121  0.999785   
3      imputed  297.201585  58.063679  0.713711  19.864258  0.998691   

   mbe (bias)  
0   -1.079455  
1   -0.092783  
2   -0.038470  
3   -0.023408  


In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
import pandas as pd
import numpy as np
import os

test_path = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/data_versions/regression/test.parquet'
model_dir = '/content/drive/MyDrive/Portfolio/Smart Meter Consumption/models/regression'

dataset_names = ['original', 'no_outliers', 'winsorized', 'imputed']
target = 'total_consumption_active_import'

test_df = pd.read_parquet(test_path)
test_df = test_df.drop(columns='data_collection_log_timestamp')
y_test = test_df[target]
X_test = test_df.drop(columns=[target])

test_results = {
    'dataset': [],
    'rmse': [],
    'mae': [],
    'mape (%)': [],
    'median_ae': [],
    'r2': [],
    'mbe (bias)': []
}

for name in dataset_names:
    print(f'Evaluating model: {name}')

    model_path = os.path.join(model_dir, f'{name}_model.json')
    model = XGBRegressor()
    model.load_model(model_path)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    medae = median_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mbe = np.mean(y_pred - y_test)

    non_zero_mask = y_test != 0
    if non_zero_mask.any():
        mape = np.mean(np.abs((y_test[non_zero_mask] - y_pred[non_zero_mask]) / y_test[non_zero_mask])) * 100
    else:
        mape = np.nan

    test_results['dataset'].append(name)
    test_results['rmse'].append(rmse)
    test_results['mae'].append(mae)
    test_results['mape (%)'].append(mape)
    test_results['median_ae'].append(medae)
    test_results['r2'].append(r2)
    test_results['mbe (bias)'].append(mbe)

test_results_df = pd.DataFrame(test_results)
print('\nTest Set Performance Summary (100 samples):')
print(test_results_df)

Evaluating model: original
Evaluating model: no_outliers
Evaluating model: winsorized
Evaluating model: imputed

Test Set Performance Summary (100 samples):
       dataset         rmse         mae  mape (%)  median_ae        r2  \
0     original   417.448189   94.631124  0.585189  17.533447  0.998030   
1  no_outliers  4029.041595  858.110376  2.520301  18.720459  0.816454   
2   winsorized  1227.738046  240.899107  2.938440  13.889954  0.982957   
3      imputed   296.656955   77.833903  0.539001  19.978027  0.999005   

   mbe (bias)  
0   10.674607  
1 -824.127695  
2 -203.375046  
3   -9.531300  


In [ ]:
print(val_results_df.columns)

Index(['model_type', 'dataset', 'rmse', 'mae', 'mape (%)', 'median_ae', 'r2',
       'mbe (bias)'],
      dtype='object')


In [ ]:
!pip install --quiet gspread gspread_dataframe

from google.colab import auth
from google.colab import drive
from gspread_dataframe import set_with_dataframe
import gspread
from google.auth import default
import pandas as pd

auth.authenticate_user()
drive.mount('/content/drive')
creds, _ = default()
gc = gspread.authorize(creds)

spreadsheet = gc.open("smart_meter_consumption_results")
worksheet = spreadsheet.sheet1

def insert_into_sheet(df, model_type_prefix, data_split):
    df['model_type'] = model_type_prefix
    df['data_split'] = data_split
    df['preprocessing'] = df['dataset'].astype(str)

    df = df.drop(columns=['dataset'], errors='ignore')
    if 'model' in df.columns:
        df = df.drop(columns=['model'])

    front_cols = ['model_type', 'data_split', 'preprocessing', 'r2']
    remaining_cols = [col for col in df.columns if col not in front_cols]
    df = df[front_cols + remaining_cols]
    df = df.drop(columns=['task_type'])

    blank_row = pd.DataFrame([[''] * len(df.columns)], columns=df.columns)
    header_row = pd.DataFrame([df.columns.tolist()], columns=df.columns)
    block = pd.concat([blank_row, header_row, df], ignore_index=True)

    next_row = len(worksheet.get_all_values()) + 1
    set_with_dataframe(worksheet, block, row=next_row, col=1, include_column_header=False)

insert_into_sheet(val_results_df, model_type_prefix='regression', data_split='val')
insert_into_sheet(test_results_df, model_type_prefix='regression', data_split='test')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
